In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
from mlxtend.frequent_patterns import association_rules, apriori

In [3]:
import pickle
with open("C:/Users/tldms/jupyter/result.pickle","rb") as fr:
    result=pickle.load(fr)

In [4]:

with open("C:/Users/tldms/jupyter/RFML.pickle","rb") as fr:
    RFML=pickle.load(fr)

In [5]:
for a in range(2):
    for b in range (2):
        for c in range(2):
            for d in range(3):
                globals()['group'+str(a)+str(b)+str(c)+str(d)] = RFML[(RFML["Recency_R"] == a) & (RFML["Frequency_F"] == b) & (RFML["Monetary_M"] == c) & (RFML["LPAY_USE"] == d) ]
                globals()['cust'+str(a)+str(b)+str(c)+str(d)] = globals()['group'+str(a)+str(b)+str(c)+str(d)]['CUST']
                globals()['transaction'+str(a)+str(b)+str(c)+str(d)] = pd.merge(result, globals()['cust'+str(a)+str(b)+str(c)+str(d)])

In [6]:
cust1010.head()

5      M000261625
6      M000350564
89     M003641896
100    M003948947
104    M004292419
Name: CUST, dtype: object

In [7]:
transaction1010.head()

,CUST,MA_FEM_DV,AGES,C_ZON_HLV,BUY_AM,COP_C,CHNL_DV,PD_C,BUY_CT,S_ZON_HLV,DE_DT,MONTH,DATE,DAY,DE_HR,CLAC_HLV_NM,CLAC_MCLS_NM,PD_NM,LPAY_USE
0,M117855953,0,30,11,759500,110,1,949,1,11,2021-01-01,1,1,4,11,여성의류,여성의류상의,여성티셔츠/탑,0
1,M117855953,0,30,11,167000,110,1,816,1,11,2021-01-13,1,13,2,20,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
2,M117855953,0,30,11,48000,110,1,816,1,11,2021-01-24,1,24,6,13,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
3,M117855953,0,30,11,126500,110,1,959,1,11,2021-02-14,2,14,6,18,여성의류,여성의류아우터,여성재킷,0
4,M117855953,0,30,11,19700,110,1,1227,1,11,2021-02-14,2,14,6,20,자동차용품,액세서리편의용품,수납/정리용품,0


## 큰손nl의 중분류 연관규칙찾는 알고리즘

큰손nl의 구매물품에 대한 연관규칙을 찾기위해서 '중분류'를 기준으로 데이터를 파악한다.<br>
중분류 카테고리 순위만 파악한 다음 대분류로 알고리즘을 돌려볼 것이다.<br>
큰손nl은 rfml기준으로 1010이라서 1010의 그룹에 속한 cust의 transaction데이터로 분석을 진행한다.<br>

여기서 cop_u도 포함되어있으니 cop_u인것을 버리기위해 조건을 건다

In [8]:
transaction1010 = transaction1010[(transaction1010['COP_C']<200)]
transaction1010

,CUST,MA_FEM_DV,AGES,C_ZON_HLV,BUY_AM,COP_C,CHNL_DV,PD_C,BUY_CT,S_ZON_HLV,DE_DT,MONTH,DATE,DAY,DE_HR,CLAC_HLV_NM,CLAC_MCLS_NM,PD_NM,LPAY_USE
0,M117855953,0,30,11,759500,110,1,949,1,11,2021-01-01,1,1,4,11,여성의류,여성의류상의,여성티셔츠/탑,0
1,M117855953,0,30,11,167000,110,1,816,1,11,2021-01-13,1,13,2,20,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
2,M117855953,0,30,11,48000,110,1,816,1,11,2021-01-24,1,24,6,13,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
3,M117855953,0,30,11,126500,110,1,959,1,11,2021-02-14,2,14,6,18,여성의류,여성의류아우터,여성재킷,0
4,M117855953,0,30,11,19700,110,1,1227,1,11,2021-02-14,2,14,6,20,자동차용품,액세서리편의용품,수납/정리용품,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131128,M058847456,0,40,10,59000,160,2,969,1,0,2021-05-29,5,29,5,11,여성의류,여성의류하의,여성청바지,0
131129,M058847456,0,40,10,39200,160,2,290,1,0,2021-07-02,7,2,4,4,남성의류,남성의류상의,남성티셔츠,0
131130,M058847456,0,40,10,158000,160,2,1718,1,0,2021-07-29,7,29,3,18,패션잡화,남성화,남성정장화,0
131131,M058847456,0,40,10,10800,160,2,1717,1,0,2021-10-23,10,23,5,18,패션잡화,남성화,남성슬리퍼,0


association rule mining에 필요한 컬럼인 'CUST','CLAC_MCLS_NM','DE_DT'를 가져와서 분석한다

In [9]:
transaction1010_m = transaction1010[['CUST','CLAC_MCLS_NM','DE_DT']]
transaction1010_m.head()

,CUST,CLAC_MCLS_NM,DE_DT
0,M117855953,여성의류상의,2021-01-01
1,M117855953,여성스포츠화,2021-01-13
2,M117855953,여성스포츠화,2021-01-24
3,M117855953,여성의류아우터,2021-02-14
4,M117855953,액세서리편의용품,2021-02-14


In [17]:
a = transaction1010.groupby(['CUST','DE_DT']).CLAC_MCLS_NM.unique().reset_index()
a['transaction'] = a.index
a = a.drop(['CUST','DE_DT'], axis=1)
a = a[['transaction','CLAC_MCLS_NM']]
a['count'] = 1
a

,transaction,CLAC_MCLS_NM,count
0,0,[스킨케어],1
1,1,"[패션액세서리, 바디케어, 식당, 여성속옷, 와인]",1
2,2,[식당],1
3,3,"[스킨케어, 메이크업]",1
4,4,[여성속옷],1
...,...,...,...
35530,35530,"[여성지갑, 남성지갑, 남성스포츠화, 여성등산/아웃도어의류, 여성가방, 여성의류상의...",1
35531,35531,"[남성등산/아웃도어의류, 요구르트, 봉지라면, 맥주, 유가공품]",1
35532,35532,[남성등산/아웃도어의류],1
35533,35533,"[남성일반스포츠의류, 생활서비스, 식당]",1


In [18]:
transaction = pd.DataFrame({'Transaction' : [], 'Item' : [], 'Count' : []})
num = 0
for i in range(35535):
    for count in range(len(a.iloc[i,1])):
        transaction.loc[num] = [i,a.iloc[i,1][count],1]
        num+=1
transaction
# 

,Transaction,Item,Count
0,0,스킨케어,1
1,1,패션액세서리,1
2,1,바디케어,1
3,1,식당,1
4,1,여성속옷,1
...,...,...,...
87530,35533,생활서비스,1
87531,35533,식당,1
87532,35534,식당,1
87533,35534,탄산음료,1


위의 transaction의 거래내역을 각 행으로 변환하는 작업의 반복문 코드가 너무 오래걸려서 <br>
pickle 파일로 변환시켰다.

In [20]:
import pickle
with open("transaction_bhnl_m.pickle","wb") as fw:
    pickle.dump(transaction, fw)

In [8]:
with open("C:/Users/tldms/jupyter/transaction_bhnl_m.pickle","rb") as fr:
    transaction=pickle.load(fr)

In [9]:
basket = transaction.pivot_table(index='Transaction', columns='Item', values='Count', aggfunc='sum').fillna(0)
basket.head()

Item,DVD/홈시어터,TV,가공유,가방세트,간식/안주형과자,간편조리,갑각류,거실가구,건강가전,건강관리용품,...,핸드/풋케어,햄/소시지,향수,향신료,헤어액세서리,헤어케어,호주산소고기,홈웨어,홍삼/인삼가공식품,화장지/티슈
Transaction,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
freq_item = apriori(basket, min_support = 0.005,use_colnames = True)
freq_item

C:\Users\tldms\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.011060,(가공유)
1,0.009005,(건면)
2,0.009427,(건어물)
3,0.007767,(견과류)
4,0.016238,(계란류)
...,...,...
154,0.005459,"(여성의류하의, 여성의류상의)"
155,0.008499,"(열매채소, 잎채소)"
156,0.005600,"(우유, 요구르트)"
157,0.005572,"(우유, 청소용품)"


In [23]:
freq_item.sort_values('support', ascending = False, inplace = True)
pd.set_option('display.max_rows', 500000)
freq_item

,support,itemsets
55,0.176333,(식당)
68,0.071141,(여성의류상의)
70,0.060785,(여성의류아우터)
106,0.059772,(푸드코트)
53,0.056508,(스낵류)
8,0.051921,(국산과일)
18,0.045167,(남성의류상의)
19,0.042437,(남성의류세트)
22,0.042184,(남성일반스포츠의류)
43,0.039679,(비스킷류)


In [24]:
rules_h = association_rules(freq_item, metric = "lift", min_threshold = 1)
rules_h.sort_values('confidence', ascending = False, inplace = True)
rules_h.head(200)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,(버섯류),(잎채소),0.014071,0.025552,0.006163,0.438000,17.141333,0.005803,1.733893
9,(열매채소),(잎채소),0.021190,0.025552,0.008499,0.401062,15.695763,0.007957,1.626960
4,(수입과일),(국산과일),0.029520,0.051921,0.010947,0.370829,7.142234,0.009414,1.506872
74,(버섯류),(열매채소),0.014071,0.021190,0.005178,0.368000,17.366375,0.004880,1.548749
23,(열매채소),(국산과일),0.021190,0.051921,0.007204,0.339973,6.547944,0.006104,1.436426
8,(잎채소),(열매채소),0.025552,0.021190,0.008499,0.332599,15.695763,0.007957,1.466599
68,(여성의류하의),(여성의류상의),0.016463,0.071141,0.005459,0.331624,4.661494,0.004288,1.389725
73,(사탕/캔디),(스낵류),0.016069,0.056508,0.005262,0.327496,5.795596,0.004354,1.402953
26,(양채류),(국산과일),0.020768,0.051921,0.006641,0.319783,6.159076,0.005563,1.393790
31,(봉지라면),(스낵류),0.019924,0.056508,0.006304,0.316384,5.598960,0.005178,1.380150


## 큰손nl의 대분류 연관규칙 찾는 알고리즘

큰손 NL에 대한 transaction으로 ARM을 진행해본 결과 VIP NL의 고객들이 주로 소비하는 품목들은 <br>
식품(채소, 스낵류 등) 이어서 품목들끼리 비슷해 추천이 의미가 없어 중분류로 마켓팅하는 것은 한계가 있다고 판단해<br>
조금 더 넓은 범위인 대분류를 이용하여 마케팅 전략을 세우려고 한다.

association rule mining에 필요한 컬럼인 'CUST','CLAC_HLV_NM','DE_DT'를 가져와서 분석한다

In [27]:
transaction1010_h = transaction1010[['CUST','CLAC_HLV_NM','DE_DT']]
transaction1010_h

,CUST,CLAC_HLV_NM,DE_DT
0,M117855953,여성의류,2021-01-01
1,M117855953,스포츠패션,2021-01-13
2,M117855953,스포츠패션,2021-01-24
3,M117855953,여성의류,2021-02-14
4,M117855953,자동차용품,2021-02-14
...,...,...,...
131135,M058847456,NULL,2021-08-14
131136,M058847456,NULL,2021-10-14
131137,M058847456,NULL,2021-11-09
131138,M058847456,NULL,2021-09-24


In [28]:
transaction1010_h = transaction1010_h.groupby(['CUST','DE_DT']).CLAC_HLV_NM.unique().reset_index()
transaction1010_h['transaction'] = transaction1010_h.index
transaction1010_h = transaction1010_h.drop(['CUST','DE_DT'], axis=1)
transaction1010_h = transaction1010_h[['transaction','CLAC_HLV_NM']]
transaction1010_h['count'] = 1
transaction1010_h

,transaction,CLAC_HLV_NM,count
0,0,[화장품/뷰티케어],1
1,1,"[패션잡화, 퍼스널케어, 테넌트/음식점, 속옷/양말/홈웨어, 주류, NULL]",1
2,2,[테넌트/음식점],1
3,3,[화장품/뷰티케어],1
4,4,[속옷/양말/홈웨어],1
...,...,...,...
42145,42145,"[스포츠패션, NULL]",1
42146,42146,"[스포츠패션, 생활/렌탈서비스, 테넌트/음식점, NULL]",1
42147,42147,[NULL],1
42148,42148,"[테넌트/음식점, 음료]",1


In [29]:
transaction1010_h_df = pd.DataFrame({'Transaction' : [], 'Item' : [], 'Count' : []})
num = 0
for i in range(42150):
    for count in range(len(transaction1010_h.iloc[i,1])):
        transaction1010_h_df.loc[num] = [i,transaction1010_h.iloc[i,1][count],1]
        num+=1
transaction1010_h_df

,Transaction,Item,Count
0,0,화장품/뷰티케어,1
1,1,패션잡화,1
2,1,퍼스널케어,1
3,1,테넌트/음식점,1
4,1,속옷/양말/홈웨어,1
...,...,...,...
84254,42146,NULL,1
84255,42147,NULL,1
84256,42148,테넌트/음식점,1
84257,42148,음료,1


In [30]:
import pickle
with open("transaction_bhnl_h.pickle","wb") as fw:
    pickle.dump(transaction1010_h_df, fw)

In [31]:
transaction1010_h_df

,Transaction,Item,Count
0,0,화장품/뷰티케어,1
1,1,패션잡화,1
2,1,퍼스널케어,1
3,1,테넌트/음식점,1
4,1,속옷/양말/홈웨어,1
...,...,...,...
84254,42146,NULL,1
84255,42147,NULL,1
84256,42148,테넌트/음식점,1
84257,42148,음료,1


In [32]:
with open("C:/Users/tldms/jupyter/transaction_bhnl_h.pickle","rb") as fr:
    transaction1010_h_df=pickle.load(fr)

In [33]:
basket_h = transaction1010_h_df.pivot_table(index='Transaction', columns='Item', values='Count', aggfunc='sum').fillna(0)
basket_h.head()

Item,NULL,가구,건강식품,건강용품,건해산물,계절가전,공구/안전용품,과일,과자,구기/필드스포츠,...,축산물,출산/육아용품,침구/수예,커피/차,컴퓨터,테넌트/음식점,패션잡화,퍼스널케어,헬스/피트니스,화장품/뷰티케어
Transaction,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
freq_item_h = apriori(basket_h, min_support = 0.01,use_colnames = True)
freq_item_h

C:\Users\tldms\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.203915,(NULL)
1,0.012479,(건강식품)
2,0.014377,(건해산물)
3,0.066833,(과일)
4,0.090866,(과자)
5,0.012266,(구기/필드스포츠)
6,0.033405,(기타(비상품))
7,0.087117,(남성의류)
8,0.023867,(냉동식품)
9,0.057556,(냉장식품)


In [35]:
freq_item_h.sort_values('support', ascending = False, inplace = True)
pd.set_option('display.max_rows', 500)
freq_item_h

,support,itemsets
0,0.203915,(NULL)
34,0.189110,(테넌트/음식점)
20,0.138482,(여성의류)
17,0.108209,(스포츠패션)
35,0.103962,(패션잡화)
4,0.090866,(과자)
7,0.087117,(남성의류)
11,0.068588,(대용식)
3,0.066833,(과일)
24,0.066002,(음료)


In [38]:
rules_h = association_rules(freq_item_h, metric = "lift", min_threshold = 1)
rules_h.sort_values('support', ascending = False, inplace = True)
rules_h.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(과일),(채소),0.066833,0.057509,0.023796,0.356053,6.191260,0.019952,1.463615
1,(채소),(과일),0.057509,0.066833,0.023796,0.413779,6.191260,0.019952,1.591835
2,(테넌트/음식점),(스포츠패션),0.189110,0.108209,0.021756,0.115042,1.063149,0.001292,1.007722
3,(스포츠패션),(테넌트/음식점),0.108209,0.189110,0.021756,0.201052,1.063149,0.001292,1.014947
4,(테넌트/음식점),(패션잡화),0.189110,0.103962,0.020996,0.111027,1.067962,0.001336,1.007948
5,(패션잡화),(테넌트/음식점),0.103962,0.189110,0.020996,0.201963,1.067962,0.001336,1.016105
6,(채소),(축산물),0.057509,0.047117,0.020617,0.358498,7.608613,0.017907,1.485394
7,(축산물),(채소),0.047117,0.057509,0.020617,0.437563,7.608613,0.017907,1.675727
8,(대용식),(과자),0.068588,0.090866,0.020000,0.291595,3.209063,0.013768,1.283353
9,(과자),(대용식),0.090866,0.068588,0.020000,0.220104,3.209063,0.013768,1.194277


## 결론 및 그에 따른 상한한도 결정하기

대분류를 기준으로 품목을 봤을 때 (테넌트/음식점을 제외하고) 의류라는 하나의 카테고리로 묶을 수 있었다<br>
(여성의류, 스포츠패션, 패션잡화, 남성의류)
그래서 의류라는 하나의 틀을 정의하고 그에 해당하는 모든 transaction을 기준으로 <br>
고객 당 한 번 구매 평균 찍어서(거래 횟수로 나누기) 금액을 구하고 그 금액보다 높은 금액으로 기준으로 상한한도를 결정하겠다.

In [42]:
import cx_Oracle
import os

LOCATION = r"C:\instantclient_21_6"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]

In [43]:
connect = cx_Oracle.connect("system", "sieun257", "localhost:1521/orcl")
cursor = connect.cursor()

중분류에서 support를 기준으로 의류와 관련된 상위 품목인 것들의 대분류 카테고리를 알아보겠다

In [44]:
clac = pd.read_sql("select * from LPAY_PD_CLAC" , con = connect)
list = ['여성의류상의','여성의류아우터','남성의류상의','남성의류세트','남성일반스포츠의류','남성스포츠화', '여성의류세트']
clac = clac.drop(['PD_C','PD_NM'],axis = 1)
for i in list:
    a = clac['CLAC_MCLS_NM']==i
    print(clac[a].drop_duplicates(),'\n')
# 대분류 파악

    CLAC_HLV_NM CLAC_MCLS_NM
948        여성의류       여성의류상의 

    CLAC_HLV_NM CLAC_MCLS_NM
957        여성의류      여성의류아우터 

    CLAC_HLV_NM CLAC_MCLS_NM
289        남성의류       남성의류상의 

    CLAC_HLV_NM CLAC_MCLS_NM
296        남성의류       남성의류세트 

    CLAC_HLV_NM CLAC_MCLS_NM
779       스포츠패션    남성일반스포츠의류 

    CLAC_HLV_NM CLAC_MCLS_NM
776       스포츠패션       남성스포츠화 

    CLAC_HLV_NM CLAC_MCLS_NM
955        여성의류       여성의류세트 



C:\Users\tldms\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


유의미한 중분류를 기준으로 봤을 때 대분류는 여성의류, 남성의류, 스포츠패션이다.<br>
그것을 포함하고 대분류에서의 연관관계를 포함해서 의류라는 하나의 틀의 기준을 세우겠다.

이제부터 앞서 정한 기준으로 큰손nl에 대한 전략을 세울 것이다.<br>
큰손nl의 transaction에서 한 번 결제할 때의 평균 구매금액을 계산해보자

In [45]:
transaction1010

,CUST,MA_FEM_DV,AGES,C_ZON_HLV,BUY_AM,COP_C,CHNL_DV,PD_C,BUY_CT,S_ZON_HLV,DE_DT,MONTH,DATE,DAY,DE_HR,CLAC_HLV_NM,CLAC_MCLS_NM,PD_NM,LPAY_USE
0,M117855953,0,30,11,759500,110,1,949,1,11,2021-01-01,1,1,4,11,여성의류,여성의류상의,여성티셔츠/탑,0
1,M117855953,0,30,11,167000,110,1,816,1,11,2021-01-13,1,13,2,20,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
2,M117855953,0,30,11,48000,110,1,816,1,11,2021-01-24,1,24,6,13,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
3,M117855953,0,30,11,126500,110,1,959,1,11,2021-02-14,2,14,6,18,여성의류,여성의류아우터,여성재킷,0
4,M117855953,0,30,11,19700,110,1,1227,1,11,2021-02-14,2,14,6,20,자동차용품,액세서리편의용품,수납/정리용품,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131135,M058847456,0,40,10,5400,410,1,4000,1,11,2021-08-14,8,14,5,12,NULL,NULL,NULL,0
131136,M058847456,0,40,10,15700,410,1,4000,1,11,2021-10-14,10,14,3,16,NULL,NULL,NULL,0
131137,M058847456,0,40,10,15300,410,1,4000,1,11,2021-11-09,11,9,1,16,NULL,NULL,NULL,0
131138,M058847456,0,40,10,9300,410,1,4000,1,11,2021-09-24,9,24,4,10,NULL,NULL,NULL,0


In [46]:
condition = transaction1010['CLAC_HLV_NM'].isin(['여성의류', '남성의류', '스포츠패션'])
transaction1010_isin = transaction1010.loc[condition]
transaction1010_isin

,CUST,MA_FEM_DV,AGES,C_ZON_HLV,BUY_AM,COP_C,CHNL_DV,PD_C,BUY_CT,S_ZON_HLV,DE_DT,MONTH,DATE,DAY,DE_HR,CLAC_HLV_NM,CLAC_MCLS_NM,PD_NM,LPAY_USE
0,M117855953,0,30,11,759500,110,1,949,1,11,2021-01-01,1,1,4,11,여성의류,여성의류상의,여성티셔츠/탑,0
1,M117855953,0,30,11,167000,110,1,816,1,11,2021-01-13,1,13,2,20,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
2,M117855953,0,30,11,48000,110,1,816,1,11,2021-01-24,1,24,6,13,스포츠패션,여성스포츠화,여성런닝/트레이닝화,0
3,M117855953,0,30,11,126500,110,1,959,1,11,2021-02-14,2,14,6,18,여성의류,여성의류아우터,여성재킷,0
5,M117855953,0,30,11,372370,110,1,949,1,11,2021-02-14,2,14,6,20,여성의류,여성의류상의,여성티셔츠/탑,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131125,M058847456,0,40,10,122600,160,2,777,1,0,2021-05-22,5,22,5,6,스포츠패션,남성스포츠화,남성런닝/트레이닝화,0
131126,M058847456,0,40,10,36800,160,2,817,1,0,2021-05-22,5,22,5,7,스포츠패션,여성스포츠화,여성스포츠샌들/슬리퍼,0
131128,M058847456,0,40,10,59000,160,2,969,1,0,2021-05-29,5,29,5,11,여성의류,여성의류하의,여성청바지,0
131129,M058847456,0,40,10,39200,160,2,290,1,0,2021-07-02,7,2,4,4,남성의류,남성의류상의,남성티셔츠,0


조건에 맞는 26344개의 transaction을 찾았다

한번 거래기록의 평균을 보려면 일단 한번 거래할때의 구매금액을 알아야한다<br>
이 한번 거래할때의 기준을 '한 사람'이 '몇 월', '몇 일', '몇 시'로 기준을 세우면 한번 거래 기록이 될 것이다.<br>
즉 (column) cust, month, date, day, de_hr을 기준으로 groupby를 하고 그때의 value값을 buy_am으로 sum하면 한번 거래할때의 구매금액이 될것이다.<br>

In [47]:
one_transaction = transaction1010_isin.groupby(['CUST', 'MONTH','DATE','DAY','DE_HR']).agg({'BUY_AM': np.sum})
one_transaction = one_transaction.reset_index()
one_transaction

,CUST,MONTH,DATE,DAY,DE_HR,BUY_AM
0,M000261625,4,3,5,13,179000
1,M000261625,10,11,0,14,588000
2,M000350564,2,25,3,19,278600
3,M000350564,5,1,5,13,2224000
4,M000350564,5,28,4,12,317300
...,...,...,...,...,...,...
14715,M997843584,10,10,6,14,1093000
14716,M997843584,10,10,6,15,79000
14717,M997843584,10,13,2,20,89000
14718,M997843584,10,17,6,12,138000


In [48]:
one_transaction['BUY_AM'].mean() 

285440.48369565216

한번 평균 금액은 28,5440원이다.